# State of the Union

## Topic Modeling With *udpipe*

### Created by Lars Bernhardsson

I used the jupyter/datascience-notebook docker image found at https://hub.docker.com/r/jupyter/datascience-notebook when making this notebook.

## R session prep

In [1]:
# If you are using the jupyter/datascience-notebook docker image to run the notebooks in this repository,
# open a terminal window and run this first:
#
#   R -e 'install.packages("udpipe", repos="https://cran.r-project.org")'

library(tidyverse)
library(udpipe)

options(stringsAsFactors = FALSE)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.2.1     ✔ purrr   0.3.3
✔ tibble  2.1.3     ✔ dplyr   0.8.4
✔ tidyr   1.0.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



## Setting up udpipe

In [2]:
en <- udpipe_download_model("english")


Visit https://github.com/jwijffels/udpipe.models.ud.2.4 for model license details



In [3]:
model <- udpipe_load_model(en$file_model)

## Data

In [4]:
if(file.exists("SOTUtexts.RData")) {
    load("SOTUtexts.RData")
} else {
    stop("Run the data notebook first")
}

## Processing

In [5]:
docs <- udpipe_annotate(model, d$Text, doc_id=d$Speech)

In [6]:
docs_df <- as.data.frame(docs)

In [7]:
docs_df %>%
    head()

,doc_id,paragraph_id,sentence_id,sentence,token_id,token,lemma,upos,xpos,feats,head_token_id,dep_rel,deps,misc
,<chr>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,1981 Reagan,1,1,"Mr. Speaker, Mr. President, distinguished Members of Congress, honored guests, and fellow citizens:",1,Mr.,Mr.,PROPN,NNP,Number=Sing,0,root,NA,SpacesBefore=\n\s\s\s\s
2,1981 Reagan,1,1,"Mr. Speaker, Mr. President, distinguished Members of Congress, honored guests, and fellow citizens:",2,Speaker,speaker,PROPN,NNP,Number=Sing,1,flat,NA,SpaceAfter=No
3,1981 Reagan,1,1,"Mr. Speaker, Mr. President, distinguished Members of Congress, honored guests, and fellow citizens:",3,",",",",PUNCT,",",NA,1,punct,NA,NA
4,1981 Reagan,1,1,"Mr. Speaker, Mr. President, distinguished Members of Congress, honored guests, and fellow citizens:",4,Mr.,Mr.,PROPN,NNP,Number=Sing,1,flat,NA,NA
5,1981 Reagan,1,1,"Mr. Speaker, Mr. President, distinguished Members of Congress, honored guests, and fellow citizens:",5,President,President,PROPN,NNP,Number=Sing,1,flat,NA,SpaceAfter=No
6,1981 Reagan,1,1,"Mr. Speaker, Mr. President, distinguished Members of Congress, honored guests, and fellow citizens:",6,",",",",PUNCT,",",NA,1,punct,NA,NA


### Extracting Top Keywords

In [8]:
speech_topics <- lapply(split(docs_df, docs_df$doc_id), function(doc) {
    keywords_rake(x = doc, term = "lemma", group = "doc_id", relevant = doc$upos %in% c("ADJ", "NOUN", "VERB"))
})

In [9]:
head(speech_topics[[1]])

,keyword,ngram,freq,rake
,<chr>,<int>,<int>,<dbl>
1,tax rate,2,3,2.810345
2,Federal Government,2,2,2.750000
3,unearned income,2,2,2.666667
4,interest rate,2,4,2.666667
5,local government,2,2,2.614907
6,block grant,2,2,2.571429


### Top Keywords by Speech

In [10]:
t(sapply(speech_topics, function(x) head(x$keyword)))

1981 Reagan,tax rate,Federal Government,unearned income,interest rate,local government,block grant
1982 Reagan,federal government,grass root,social service,military force,local community,tax increase
1983 Reagan,foreign policy,international trade,national debt,interest rate,majority leader,technological leader
1984 Reagan,increase tax,economic recovery,bipartisan cooperation,down payment,underground economy,personal stake
1985 Reagan,federal government,tax simplification,american people,give,fellow citizen,assistance
1986 Reagan,true reform,federal budget,american people,national security,human spirit,next decade
1987 Reagan,national interest,welfare reform,arm reduction,new job,third century,fellow citizen
1988 Reagan,Federal Government,budget process,national security,balanced budget,arm reduction,free market
1989 BushSr,new program,nuclear weapon,best judgment,growth,child care,lease sale
1990 BushSr,record high,new initiative,american worker,new world,american people,american idea
1991 BushSr,Sdi program,federal government,dead wrong,fit tribute,american people,define hour


In [11]:
topics_by_speech <- bind_rows(speech_topics, .id="Speech")

## Specific Keywords

In [12]:
## Find the top mentions of keywords

find_topic <- function(x) {
    topics_by_speech %>%
        filter(str_detect(keyword, fixed(x))) %>%
        arrange(desc(rake)) %>%
        head()
}

In [13]:
find_topic("health")

,Speech,keyword,ngram,freq,rake
,<chr>,<chr>,<int>,<int>,<dbl>
1,2004 GWBush,health care,2,3,4.125000
2,2007 GWBush,health insurance,2,4,4.033613
3,2009 Obama,health care,2,5,3.631579
4,2004 GWBush,health insurance,2,2,3.625000
5,1994 Clinton,health care,2,7,3.536585
6,2001 GWBush,health care,2,3,3.383333


In [14]:
find_topic("drug")

,Speech,keyword,ngram,freq,rake
,<chr>,<chr>,<int>,<int>,<dbl>
1,2000 Clinton,prescription drug,2,2,3.1666667
2,2019 Trump,prescription drug,2,2,2.5000000
3,2020 Trump,drug,1,2,1.4444444
4,1994 Clinton,drug treatment,2,2,1.3030303
5,2003 GWBush,drug,1,3,1.1000000
6,1989 BushSr,drug,1,8,0.7692308


In [15]:
find_topic("immigration")

,Speech,keyword,ngram,freq,rake
,<chr>,<chr>,<int>,<int>,<dbl>
1,2019 Trump,immigration system,2,2,2.8666667
2,2019 Trump,illegal immigration,2,2,2.8000000
3,2008 GWBush,immigration,1,2,0.3333333
4,2015 Obama,immigration,1,2,0.0000000


In [16]:
find_topic("gun")

,Speech,keyword,ngram,freq,rake
,<chr>,<chr>,<int>,<int>,<dbl>
1,2000 Clinton,keep gun,2,2,2.0600000
2,1994 Clinton,own gun,2,2,1.9821429
3,2013 Obama,gun violence,2,2,1.6666667
4,2000 Clinton,gun,1,4,1.5600000
5,1994 Clinton,gun,1,2,0.8571429
6,2013 Obama,gun,1,2,0.8333333


In [17]:
find_topic("tax")

,Speech,keyword,ngram,freq,rake
,<chr>,<chr>,<int>,<int>,<dbl>
1,1998 Clinton,tax credit,2,2,4.000000
2,1998 Clinton,tax cut,2,2,3.700000
3,2000 Clinton,tax credit,2,4,3.692308
4,2013 Obama,tax reform,2,2,3.371429
5,2005 GWBush,tax code,2,2,3.366667
6,2010 Obama,tax credit,2,2,3.023810


In [18]:
find_topic("bank")

,Speech,keyword,ngram,freq,rake
,<chr>,<chr>,<int>,<int>,<dbl>
1,2009 Obama,bank,1,4,1.00
2,2010 Obama,bank,1,4,1.00
3,2012 Obama,bank,1,3,0.25
4,1994 Clinton,bankruptcy,1,2,0.00


In [19]:
find_topic("security")

,Speech,keyword,ngram,freq,rake
,<chr>,<chr>,<int>,<int>,<dbl>
1,2008 GWBush,energy security,2,2,2.466667
2,1988 Reagan,national security,2,2,2.450000
3,2010 Obama,national security,2,3,2.333333
4,1986 Reagan,national security,2,2,1.880952
5,2013 Obama,security,1,2,1.700000
6,2007 GWBush,own security,2,2,1.443182


In [20]:
find_topic("terror")

,Speech,keyword,ngram,freq,rake
,<chr>,<chr>,<int>,<int>,<dbl>
1,2002 GWBush,terrorist camp,2,2,2.375000
2,1988 Reagan,nuclear terror,2,2,2.142857
3,2006 GWBush,terror network,2,2,1.833333
4,2006 GWBush,terrorist,1,2,1.454545
5,2013 Obama,counterterrorism effort,2,2,1.400000
6,2005 GWBush,harbor terrorist,2,2,1.250000


In [21]:
find_topic("future")

,Speech,keyword,ngram,freq,rake
,<chr>,<chr>,<int>,<int>,<dbl>
1,1982 Reagan,near future,2,2,2.111111
2,1993 Clinton,economic future,2,2,1.867133
3,2013 Obama,future generation,2,2,1.500000
4,1989 BushSr,better future,2,2,1.466667
5,1994 Clinton,own future,2,2,1.458333
6,2016 Obama,better future,2,2,1.351648


In [22]:
find_topic("illegal")

,Speech,keyword,ngram,freq,rake
,<chr>,<chr>,<int>,<int>,<dbl>
1,2020 Trump,illegal alien,2,3,3.3
2,1995 Clinton,illegal alien,2,2,3.0
3,2019 Trump,illegal immigration,2,2,2.8


In [23]:
find_topic("prescription")

,Speech,keyword,ngram,freq,rake
,<chr>,<chr>,<int>,<int>,<dbl>
1,2000 Clinton,prescription drug,2,2,3.166667
2,2019 Trump,prescription drug,2,2,2.500000


In [24]:
find_topic("insurance")

,Speech,keyword,ngram,freq,rake
,<chr>,<chr>,<int>,<int>,<dbl>
1,2007 GWBush,health insurance,2,4,4.033613
2,2004 GWBush,health insurance,2,2,3.625000
3,1994 Clinton,health insurance,2,4,3.253049
4,1999 Clinton,health insurance,2,2,3.150000
5,2014 Obama,unemployment insurance,2,2,3.071429
6,1998 Clinton,health insurance,2,2,2.933333


In [25]:
find_topic("employ")

,Speech,keyword,ngram,freq,rake
,<chr>,<chr>,<int>,<int>,<dbl>
1,2020 Trump,unemployment rate,2,4,3.5000000
2,2014 Obama,unemployment insurance,2,2,3.0714286
3,1983 Reagan,unemployment,1,2,2.0000000
4,1983 Reagan,term unemployed,2,2,1.6666667
5,1995 Clinton,employ people,2,2,1.6027397
6,1982 Reagan,unemployment,1,3,0.7142857


In [26]:
find_topic("industry")

,Speech,keyword,ngram,freq,rake
,<chr>,<chr>,<int>,<int>,<dbl>
1,1981 Reagan,major industry,2,2,2.3214286
2,2012 Obama,industry,1,3,1.0000000
3,2000 Clinton,industry,1,2,0.6000000
4,1981 Reagan,industry,1,3,0.5714286


In [27]:
find_topic("war")

,Speech,keyword,ngram,freq,rake
,<chr>,<chr>,<int>,<int>,<dbl>
1,1998 Clinton,global warming,2,2,2.428571
2,1997 Clinton,cold war,2,5,2.166667
3,1992 BushSr,war begin,2,2,1.833333
4,1992 BushSr,cold war,2,2,1.833333
5,1995 Clinton,cold war,2,6,1.750000
6,1994 Clinton,reward work,2,3,1.360000


In [28]:
find_topic("border")

,Speech,keyword,ngram,freq,rake
,<chr>,<chr>,<int>,<int>,<dbl>
1,2019 Trump,southern border,2,4,2.5476190
2,2020 Trump,southern border,2,2,2.0476190
3,2019 Trump,border,1,2,1.2142857
4,2020 Trump,border,1,3,0.7142857
5,2005 GWBush,border,1,2,0.6666667
6,2006 GWBush,border,1,2,0.5000000


In [29]:
find_topic("impeach")

Speech,keyword,ngram,freq,rake
<chr>,<chr>,<int>,<int>,<dbl>
